# Exceptions in Jupyterlab

Normally when exception is rased in a cell, Jupyter does not continue executing the following cells. It looks like that if we do not catch the exception all the methods printing it do not obey setting of `sys.tracebacklimit` which normally allows us to limit the printing of the traceback.

In [1]:
# initialization
import sys

## Cell tag

In Jupyterlab we can set *Cell Tag* `raises-exception` on a cell to prevent this behaviour.
* https://github.com/jupyterlab/jupyterlab/issues/2412

How to set cell tags? Select the cell. Open *Property Inspector* (right sidebar). Add the tag there in the section *Cell Tags*. The cell below has this tag set.

In [2]:
sys.tracebacklimit = None  # default
assert False, 'Testing exception'

AssertionError: Testing exception

In [3]:
print('execution continues')

execution continues


In [4]:
sys.tracebacklimit = 0  # do not print traceback
assert False, 'Testing exception'

AssertionError: Testing exception

## Cell magic

In [5]:
%%script --no-raise-error python

sys.tracebacklimit = None  # default
assert False, 'Testing exception'

Traceback (most recent call last):
  File "<stdin>", line 2, in <module>
NameError: name 'sys' is not defined


In [6]:
print('execution continues')

execution continues


In [7]:
%%script --no-raise-error python

sys.tracebacklimit = 0  # do not print traceback
assert False, 'Testing exception'

Traceback (most recent call last):
  File "<stdin>", line 2, in <module>
NameError: name 'sys' is not defined


## Explicitly catch the exception

In [15]:
import traceback

sys.tracebacklimit = None  # default
try:
    assert False, 'Testing exception'
except AssertionError:     # we can catch more (or all) exceptions
    traceback.print_exc()

Traceback (most recent call last):
  File "/tmp/ipykernel_3019/3427197396.py", line 5, in <cell line: 4>
    assert False, 'Testing exception'
AssertionError: Testing exception


In [9]:
print('execution continues')

execution continues


`traceback.print_exc()` obeys setting of `sys.tracebacklimit`.

In [26]:
import traceback

sys.tracebacklimit = 0  # do not print traceback
try:
    assert False, 'Testing exception'
except AssertionError:     # we can catch more (or all) exceptions
    traceback.print_exc()

AssertionError: Testing exception


Sources:
* https://docs.python.org/3/library/traceback.html#traceback.format_exc

In [42]:
import traceback

def print_exc_short(file=sys.stderr) -> None:
    """Print only the exception and line number."""
    print(
            f'{traceback.format_exc(limit=0).rstrip()}; '
            f'line {sys.exc_info()[-1].tb_lineno}', file=file)

sys.tracebacklimit = None  # default
try:
    assert False, 'Testing exception'
except AssertionError:     # we can catch more (or all) exceptions
    print_exc_short()

AssertionError: Testing exception; line 11


In [30]:
sys.tracebacklimit = None  # default
try:
    assert False, 'Testing exception'
except AssertionError as exception:     # we can catch more (or all) exceptions
    print(repr(exception), file=sys.stderr)

AssertionError('Testing exception')


## Selected solution

### Todo
It seems that Jupyter uses full buffering for output. I do not know how to change it.

In vanilla Python:
```Python
>>> type(sys.stdout)
<class '_io.TextIOWrapper'>
>>> type(sys.stderr)
<class '_io.TextIOWrapper'>
```
In Jupyter:

In [117]:
type(sys.stdout)

ipykernel.iostream.OutStream

In [118]:
type(sys.stderr)

ipykernel.iostream.OutStream

In vanilla Python we can reconfigure the buffering using [`reconfigure()`](https://docs.python.org/3/library/io.html#io.TextIOWrapper.reconfigurehttps://docs.python.org/3/library/io.html#io.TextIOWrapper.reconfigure) method:
```Python
sys.stdout.reconfigure(line_buffering=True)
sys.stderr.reconfigure(line_buffering=True)
```
Unfortunately `ipykernel.iostream.OutStream` does not have the method `.reconfigure()`: `OutStream has no attribute reconfigure`. See [How to change the buffering mode of sys.stdout and sys.stderr?How to change the buffering mode of sys.stdout and sys.stderr?](https://discourse.jupyter.org/t/how-to-change-the-buffering-mode-of-sys-stdout-and-sys-stderr/14484https://discourse.jupyter.org/t/how-to-change-the-buffering-mode-of-sys-stdout-and-sys-stderr/14484)

In [132]:
import sys
import traceback
import contextlib
from typing import Iterable, Iterator

def print_exc_short(file=sys.stderr) -> None:
    """Print the curent exception with only message and line number.

    Format of the printed message is:
        ExceptionType: exception_message; line line_number

    The function is designed to be useable with IPython (and Jupyter) or when
    stdout and/or stderr use full buffering. It takes care of the correct
    interleaving of the two streams.
    
    Args:
        file: stream to print the exception to

    Todo:
        * Optionally print module/file name for other uses than
            IPython/Jupyter.
    """
    # Workaround for non-reconfigurable full buffering in IPython
    # ipykernel.iostream.OutStream does not have the method .reconfigure()
    for stream in (sys.stdout, sys.stderr):
        if stream != file:
            stream.flush()
    print(
            # traceback.format_exc() adds a trailing newline, remove it:
            f'{traceback.format_exc(limit=0).rstrip()}; '
            # sys.exc_info()[2].tb_lineno contains line inside the try: block
            # We need to go to the lowest frame where the exception happened:
            f'line {traceback.extract_tb(sys.exc_info()[2])[-1].lineno}',
            file=file, flush=True)

@contextlib.contextmanager
def suppress_print(
        exceptions: BaseException | Iterable[BaseException], file=sys.stderr
        ) -> Iterator[None]:
    """Suppress the listed exception types and print them concisely.

    The function print_exc_short() is used to print the exception.

    Args:
        file: stream to print the exception to
    """
    try:
        yield
    except exceptions:
        print_exc_short(file)

In [133]:
with suppress_print(AssertionError):
    assert False, 'This is a testing assert'
    print('This will not execute')
print('Here we continue after assert')
with suppress_print(Exception):
    x = 1 / 0
    print('This will not execute')
print('Here we continue after division')

AssertionError: This is a testing assert; line 2


Here we continue after assert


ZeroDivisionError: division by zero; line 6


Here we continue after division


In [139]:
import ipykernel
type(sys.stdout) == ipykernel.iostream.OutStream

True